In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import calendar
import numpy as np
import pandas as pd
import xgboost as xgb

In [3]:
train = pd.read_csv('/Users/tianyixia/dev/Qishi_ML_adv/kaggle/walmart/data/train.csv', dtype={'Upc': str, 'FinelineNumber': str})

In [4]:
test = pd.read_csv('/Users/tianyixia/dev/Qishi_ML_adv/kaggle/walmart/data//test.csv', dtype={'Upc': str, 'FinelineNumber': str})

In [5]:
target_column = train.columns[0]
target_column

'TripType'

In [6]:
data = train.append(test)[train.columns]

In [7]:
feature_columns = []

In [8]:
data['NumberItems'] = data.groupby('VisitNumber')['VisitNumber'].transform('count')
feature_columns += ['NumberItems']

In [9]:
day_number_by_name = {d: n for n, d in enumerate(calendar.day_name)}
data['WeekdayNumber'] = data['Weekday'].map(day_number_by_name)
data['IsWeekday'] = data['WeekdayNumber'] < 5
feature_columns += ['WeekdayNumber', 'IsWeekday']

In [10]:
data['LenUpc'] = data['Upc'].fillna('').apply(len)
feature_columns += ['LenUpc']

In [11]:
data['TotalScanCount'] = data.groupby('VisitNumber')['ScanCount'].transform('sum')
feature_columns += ['ScanCount', 'TotalScanCount']

In [12]:
departments = list(data['DepartmentDescription'].unique())
feature_columns += departments

for department in departments:
    data[department] = data['DepartmentDescription'] == department

In [13]:
data['LenFinelineNumber'] = data['FinelineNumber'].fillna('').apply(len)
feature_columns += ['FinelineNumber']

In [17]:
train_mask = data[target_column].notnull()
X_train = data[train_mask][feature_columns].as_matrix()
y_train = data[train_mask][target_column].as_matrix()
X_test = data[~train_mask][feature_columns].as_matrix()
sample_weight = 1 / data['NumberItems']

In [20]:
gbm = xgb.XGBClassifier().fit(X_train, y_train, sample_weight=sample_weight)

In [21]:

predictions = gbm.predict_proba(X_test)

In [22]:
columns = ['TripType_{}'.format(int(t)) for t in gbm._le.inverse_transform(range(predictions.shape[1]))]

In [23]:
submission = pd.concat([test['VisitNumber'], pd.DataFrame(predictions, columns=columns)], axis=1)

In [24]:
submission = submission.groupby('VisitNumber').mean()

In [25]:
submission.to_csv('/Users/tianyixia/dev/Qishi_ML_adv/kaggle/walmart/data/submission.csv')